In [1]:
import numpy as np

In [2]:
def read_text(path):
  with open(path) as f:
    text = f.read()
  return text

In [3]:
def sep_punc(text):
  toks = []
  for token in nlp(text):
    if token.text not in '\n\n\n!"#$%&()*+,-./:;<  --=>?@[\\]^_`{|}~\t\n':
      toks.append(token.text.lower())   
  return toks   

In [4]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [5]:
# path = "/content/moby_dick_four_chapters.txt"
path = '/content/hp p1.txt'
file_text = read_text(path)
tokens = sep_punc(file_text)
# print(tokens )

In [6]:
train_len = 40 + 1
text_sequences = []
for i in range(train_len , len(tokens)):
  seq = tokens[i - train_len : i]
  text_sequences.append(seq)


In [7]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters = '\n\n\n!"#$%&()*+,-./:;<  --=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
# print(sequences)

In [8]:
ind_word = tokenizer.index_word

In [ ]:
ind_word

In [10]:
vocab_size = len(tokenizer.word_counts)
vocab_size

5860

In [12]:
from tensorflow.keras.utils import to_categorical
sequences = np.array(sequences)
X = sequences[: , :-1]
y = sequences[: , -1]
y = to_categorical(y , num_classes = vocab_size + 1) # + 1 for holding zeroth class

seq_len = X.shape[1]
X.shape

(81197, 40)

In [15]:
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM , Dropout, Flatten

In [18]:
def create_model(vocab_size , seq_len):
  model = Sequential()
  model.add(Embedding(input_dim = vocab_size , output_dim= seq_len * 2, input_length = seq_len))
  model.add(LSTM(units= seq_len * 2 , return_sequences= True))
  # model.add(LSTM(units = seq_len*2))
  model.add(Flatten())
  model.add(Dense(seq_len , activation = 'relu'))
  model.add(Dropout(0.2))
  model.add(Dense(vocab_size , activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])
  model.summary()
  return model

In [19]:
model = create_model(vocab_size + 1 , seq_len)
model.fit(X, y , batch_size = 100 , epochs = 100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 80)            468880    
_________________________________________________________________
lstm (LSTM)                  (None, 40, 80)            51520     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                128040    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5861)              240301    
Total params: 888,741
Trainable params: 888,741
Non-trainable params: 0
________________________________________________

In [20]:
from keras.preprocessing.sequence import pad_sequences
def generate_text(model , tokenizer , seq_len , seed_text, no_of_words):
  output_text = []
  input_text = seed_text

  for i in range(no_of_words):
    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = pad_sequences([encoded_text] , maxlen = seq_len , truncating = 'pre')

    ind = np.argmax(model.predict(pad_encoded) , axis = 1)[0]
    # type(ind)
    word = tokenizer.index_word[ind]
    input_text += ' ' + word

    output_text.append(word)

  return ' '.join(output_text)

In [21]:
import random
r_p = random.randint(0 , len(text_sequences))
rand_text = text_sequences[r_p]
seed_text = ' '.join(rand_text)

# seed_text
new = generate_text(model , tokenizer ,seq_len , seed_text , 30)
# print(seed_text + "\n" + new )
seed_text

'obviously in one of the towers they found their beds at last five four posters hung with deep red velvet curtains their trunks had already been brought up too tired to talk much they pulled on their pajamas and fell into'

In [22]:
# seed_text = 'harry did not wanted to join the slytherine he knew about people who were particularly bad and belonged to slytherine'
generate_text(model , tokenizer , seq_len , seed_text , 20)

"bed great food is n't it ron muttered to harry through the shadow and his face harry was moving lee"